# 앙상블과 DropOut

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

## 참고 자료
https://untitledtblog.tistory.com/156

# 앙상블 학습
- 일련의 예측기(즉,분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있는 방식
- 일련의 에측기 : 앙상블

# 1. 가장 기본적인 앙상블 방법: 투표 기반 앙상블
강력한 성능에 비해 의외로 앙상블 방법의 개념은 매우 단순하다. 앙상블 개념의 핵심 개념은 다양한 종류의 여러 모델들을 결합하여 더 좋은 모델를 만드는 것이다. 앙상블 방법의 종류는 모델들을 어떻게 결합할 것인지에 의해 결정된다. 가장 단순한 앙상블 방법으로는 아래 그림과 같이 투표를 기반으로 base 모델들을 결합하여 최종 모델를 만드는 것이 있다.


<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FbKOy8L%2FbtqAnvTpKWZ%2Fftj57mimws0fvsb22yvbo0%2Fimg.png">


# DropOut
- 뉴런의 연결을 임의로 삭제하는 것.<br>훈련할 때 임의의 뉴런을 골라 삭제하여 신호를 전달하지 않게 한다. 


언뜻 생각하면 그냥 학습을 덜 하는 것이고 실제로도 어느정도는 맞는 말이다. 일정 확률로 뉴런을 사용하지 않다보면 '영향력이 지나치게 강한' 뉴런이 무시될 수도 있다. 영향력이 지나치게 강하다는 것은 학습 데이터에 대해서 확신이 강하다는 뜻으로 볼 수 있다. 그런 뉴런이 많으면 분명 학습 데이터에서 문제를 푸는 것은 잘 하지만, 그럴 수록 실제 데이터에 대한 감은 떨어질 수 있다.

<비유><br>
인간으로 치자면 이미 익숙한 유형의 문제를 접했을 때 바로바로 푸는 게 아니라 문제를 꼼꼼히 읽어보는 것에 비유할 수 있다. 아는 문제라고 덥석 무는 게 아니라 혹시 내가 생각 못한 부분이 있을 수도 있기 때문이다.

<장점><br>
드롭아웃의 가장 좋은 점은 무엇보다도 간단하다는 것이다. 이 외에도 과적합을 방지하기 위한 기법은 많이 있지만, 개념적으로 보나 실제 구현으로 보나 굉장히 간단한데도 효과를 보기가 쉬워서 쓰기 편하다.

<img src="http://opennmt.net/OpenNMT/img/dropout.jpg">

In [6]:
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None
    
    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)
    
    def backward(self,dout):
        return dout * self.mask
from scratch.common.multi_layer_net_extend import MultiLayerNetExtend
from scratch.common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# 드롭아웃 사용 유무와 비울 설정 ========================
use_dropout = True  # 드롭아웃을 쓰지 않을 때는 False
dropout_ratio = 0.2
# ====================================================

network = MultiLayerNetExtend(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100],
                              output_size=10, use_dropout=use_dropout, dropout_ration=dropout_ratio)
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=301, mini_batch_size=100,
                  optimizer='sgd', optimizer_param={'lr': 0.01}, verbose=True)
trainer.train()

train_acc_list, test_acc_list = trainer.train_acc_list, trainer.test_acc_list

# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()



AttributeError: module 'tensorflow' has no attribute 'MultiLayerNetExtend'

In [ ]:
import 